In [9]:
import pandas as pd
import numpy as np
from scipy.interpolate import make_smoothing_spline

import livef1

In [4]:
season = 2023
dfs = []
season_df = livef1.get_season(season)
if len(season_df.meetings_table[season_df.meetings_table["Meeting Circuit Shortname"] == "Jeddah"]) > 0:

    sess_table = livef1.get_meeting(
        season,
        'Jeddah',
    ).sessions_table
    sess_table = sess_table[~sess_table['path'].isnull()]

    for sess_iden in sess_table['session_name'].unique():
        session = livef1.get_session(
            season,
            meeting_identifier='Jeddah',
            session_identifier=sess_iden,
        )
        session.generate(),
        if hasattr(session, 'laps'):
            if len(session.laps) > 0:

                laps = session.laps.copy()
                filtered_laps = laps[laps["in_pit"].isnull() & laps["pit_out"].isnull() & ~laps["isDeleted"]]
                filtered_laps["tuple"] = list(zip(filtered_laps.DriverNo, filtered_laps.lap_number))
                session.carTelemetry["tuple"] = list(zip(session.carTelemetry.DriverNo, session.carTelemetry.lap_number))
                car_tel = session.carTelemetry[session.carTelemetry["tuple"].isin(filtered_laps["tuple"])]

                dfs.append(car_tel)

tel_df = pd.concat(dfs)

tel_df_processed = tel_df.copy()
tel_df_processed["Distance"] = tel_df_processed["Distance"].round()
tel_df_processed = tel_df_processed[["Distance","X","Y","Z"]].groupby("Distance").mean().reset_index()
tel_df_processed = tel_df_processed[tel_df_processed["Distance"] > 0].reset_index(drop=True)

17:22:29 - Selected meeting/session is:
	Meeting Offname : FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2023
	Meeting Name : Saudi Arabian Grand Prix
	Meeting Circuit Shortname : Jeddah
17:22:29 - Got the meeting.
17:22:30 - Selected meeting/session is:
	Meeting Offname : FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2023
	Meeting Name : Saudi Arabian Grand Prix
	Meeting Circuit Shortname : Jeddah
17:22:30 - Got the meeting.
17:22:30 - Selected meeting/session is:
	Session Name : Practice 1
17:22:30 - The session was received successfully.
17:22:30 - Fetching drivers.
17:22:31 - Fetching data : 'TimingData'
17:22:31 - Fetching data : 'CarData.z'
17:22:31 - Fetching data : 'Position.z'
17:22:31 - Fetching data : 'SessionData'
17:22:31 - Fetching data : 'SessionStatus'
17:22:31 - Fetching data : 'RaceControlMessages'
17:22:31 - 'SessionData' has been fetched and parsed
17:22:31 - 'RaceControlMessages' has been fetched and parsed
17:22:31 - 'SessionStatus' has been fetched and parsed
17:22:32 - 'Timin

NameError: name 'pd' is not defined

In [13]:
tel_df_processed

,Utc,DriverNo,SessionKey,timestamp,rpm,speed,n_gear,throttle,brake,drs,SessionKey_pos,timestamp_pos,Status,X,Y,Z,lap_number,Distance,tuple
9475,2023-03-17 13:50:38.122,1,7772.0,0 days 00:35:38.118000,10759.000000,304.000000,8.0,100.0,0.0,12.0,NaN,NaT,NaN,-1307.323979,262.131380,118.000099,2.0,-3.0,"(1, 2.0)"
9476,2023-03-17 13:50:38.124,1,7772.0,0 days 00:35:38.120000,10786.666667,304.666667,8.0,100.0,0.0,12.0,7772.0,0 days 00:35:38.340000,OnTrack,-1308.000000,264.000000,118.000000,2.0,-2.0,"(1, 2.0)"
9477,2023-03-17 13:50:38.344,1,7772.0,0 days 00:35:38.340000,10814.333333,305.333333,8.0,100.0,0.0,12.0,7772.0,0 days 00:35:38.340000,OnTrack,-1390.000000,487.000000,118.000000,2.0,16.0,"(1, 2.0)"
9478,2023-03-17 13:50:38.442,1,7772.0,0 days 00:35:38.438000,10842.000000,306.000000,8.0,100.0,0.0,12.0,NaN,NaT,NaN,-1417.145862,560.993702,118.056608,2.0,25.0,"(1, 2.0)"
9479,2023-03-17 13:50:38.664,1,7772.0,0 days 00:35:38.660000,10872.500000,307.000000,8.0,100.0,0.0,12.0,7772.0,0 days 00:35:39.581000,OnTrack,-1466.000000,695.000000,118.000000,2.0,44.0,"(1, 2.0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702612,2023-03-19 18:26:30.170,81,7779.0,0 days 02:25:30.084000,11535.000000,289.000000,7.0,98.0,0.0,0.0,NaN,NaT,NaN,-1144.100628,-212.933311,117.696201,50.0,6049.0,"(81, 50.0)"
702613,2023-03-19 18:26:30.482,81,7779.0,0 days 02:25:30.396000,11457.000000,289.500000,7.0,98.0,0.0,0.0,7779.0,0 days 02:25:30.396000,OnTrack,-1211.000000,-14.000000,118.000000,50.0,6074.0,"(81, 50.0)"
702614,2023-03-19 18:26:30.490,81,7779.0,0 days 02:25:30.404000,11379.000000,290.000000,7.0,98.0,0.0,0.0,NaN,NaT,NaN,-1212.364398,-10.001829,118.003604,50.0,6074.0,"(81, 50.0)"
702615,2023-03-19 18:26:30.782,81,7779.0,0 days 02:25:30.696000,11447.500000,290.000000,7.0,98.0,0.0,0.0,7779.0,0 days 02:25:31.496000,OnTrack,-1259.000000,124.000000,118.000000,50.0,6098.0,"(81, 50.0)"


In [14]:
tel_df = pd.concat(dfs)

tel_df_processed = tel_df.copy()
tel_df_processed["Distance"] = tel_df_processed["Distance"].round()
tel_df_processed = tel_df_processed[["Distance","X","Y","Z","speed"]].groupby("Distance").mean().reset_index()
tel_df_processed = tel_df_processed[tel_df_processed["Distance"] > 0].reset_index(drop=True)

In [15]:
df_dict = {}
for col in ["X", "Y", "Z", "speed"]:
    x = tel_df_processed.sort_values("Distance", ascending=True)['Distance'].values
    y = tel_df_processed.sort_values("Distance", ascending=True)[col].values

    xnew = np.arange(
        x[0],
        x[-1],
        1
    )

    spl = make_smoothing_spline(x, y)
    ynew = spl(xnew)

    df_dict[col] = ynew
df_dict["Distance"] = xnew
circuit_df = pd.DataFrame(df_dict)

In [16]:
circuit_df

,X,Y,Z,speed,Distance
0,-1323.192169,304.113004,117.218532,292.451603,1.0
1,-1326.763214,313.802700,117.232950,292.590498,2.0
2,-1330.333637,323.490870,117.247490,292.729759,3.0
3,-1333.902455,333.175114,117.262305,292.870186,4.0
4,-1337.468921,342.853583,117.277452,293.012434,5.0
...,...,...,...,...,...
6164,-1402.758901,521.400744,118.040976,290.722312,6165.0
6165,-1405.423166,528.638574,118.043625,290.521470,6166.0
6166,-1408.087990,535.877775,118.046289,290.327280,6167.0
6167,-1410.753232,543.118005,118.048964,290.138080,6168.0


In [17]:
import plotly.express as px

fig = px.scatter(
    circuit_df,
    x="X",
    y="Y",
    color="speed",
    color_continuous_scale=px.colors.sequential.Plasma,
    title="Jeddah Corniche Circuit Telemetry",
    labels={"X": "X", "Y": "Y", "speed": "Speed (km/h)"},
)

fig.update_traces(marker=dict(size=5))

fig.show()